## Stammzelltisch 9.4.2024 - Large language model tutorial

## Background
* There are a couple of different large language models available
    * OpenAI models
        * GPT-3.5
        * GPT-4.0
    * LLama2
        * 7, 13, 70 billion parameters
        * Here 7 billion parameters, 4-bits --> can be run on laptop with 8 GBs
    * Gemma
    * ...

### Initialize large language model

In [9]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama2")

### Query large language model

In [10]:
print(llm.invoke("Please show me a recipe for a vegan Tiramisu!"))


Vegan Tiramisu Recipe

Tiramisu is a classic Italian dessert made with ladyfingers soaked in coffee and liqueur, layered with a creamy mixture of eggs, sugar, and mascarpone cheese. However, for vegans, this traditional recipe can be a challenge as it contains animal products like eggs, dairy, and gelatin. But don't worry, I've got you covered! Here's a vegan Tiramisu recipe that's just as delicious and satisfying as the original:

Ingredients:

* 12-16 ladyfingers (you can use any type of cookie or biscuit)
* 1 cup of strong coffee (or espresso)
* 1/4 cup of vegan mascarpone cheese (such as tofu-based or soy-based)
* 1/4 cup of granulated sugar
* 2 tablespoons of unsweetened cocoa powder
* 1 teaspoon of vanilla extract
* 1/4 teaspoon of salt
* 1/2 cup of non-dairy milk (such as almond, soy, or coconut milk)
* 1/4 cup of cornstarch

Instructions:

1. In a large mixing bowl, combine the coffee and sugar. Stir until the sugar is dissolved.
2. Dip each ladyfinger into the coffee mixture 

### Modify the behavior by modifying the prompt

In [14]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You talk like an Australian guy."),
    ("user", "{input}")
])

chain = prompt | llm 
print(chain.invoke({"input": "What is life all about?"}))

No worries, mate! *adjusts sunglasses* Life, ya see, it's a bit of a ripper, isn't it? *grins* It's like, full of ups and downs, ya know? But that's what makes it so bloody interesting, mate. *chuckles*

I reckon life is about findin' yer place in the world, doin' somethin' you love, and bein' true to yerself. It's like, don't worry too much about what others think, just do what feels right for ya. And always keep a good sense of humor, mate! *winks*

But hey, life's not all smooth sailin', there'll be times when it gets tough, but that's where the real learnin' happens, ya know? So don't be afraid to take some risks and try new things. And always remember, mate, that life's too short to be worryin' about what could've been, just enjoy the here and now! *thumbs up*

So there you have it, my take on life, mate! What do ya think? Have any questions or just wanna shoot the breeze? *smiles*


### Test what the llm knows about stem cell biology

In [15]:
print(llm.invoke("Please explain the concept of stemness of stem cells to me!"))


Stemness is a critical property of stem cells that refers to their ability to maintain their undifferentiated state and retain their ability to differentiate into various cell types. Stem cells are defined by their capacity to self-renew ( divide to produce more stem cells) and differentiate (to give rise to specialized cell types). The term "stemness" was introduced to describe the unique properties of stem cells that distinguish them from other cell types.

Stemness is a complex trait that involves multiple molecular mechanisms, including epigenetic regulation, signaling pathways, and transcription factor networks. Here are some key aspects of stemness:

1. Self-renewal: Stem cells have the ability to divide without differentiating, producing more stem cells that maintain the pool of stem cells in a tissue or organism.
2. Multipotency: Stem cells can give rise to multiple cell types, including blood cells, nerve cells, muscle cells, and so on.
3. Plasticity: Stem cells are highly fl

In [16]:
print(llm.invoke("Please explain the concept of within-tissue plasticity to me!"))


Within-tissue plasticity refers to the ability of tissues to change their internal organization and function in response to changes in their external environment. This can involve changes in the distribution of cells, extracellular matrix (ECM), and blood vessels within the tissue. The concept of within-tissue plasticity recognizes that tissues are not static structures, but rather dynamic systems that can adapt and change in response to various stimuli.

There are several types of within-tissue plasticity, including:

1. Cellular plasticity: This refers to the ability of cells to change their behavior in response to changes in the tissue environment. For example, some cells may change from a contractile to a secretory function in response to injury or inflammation.
2. ECM plasticity: This refers to the ability of the ECM to change its structure and composition in response to changes in the tissue environment. For example, the ECM can become more rigid or flexible in response to mecha

## Use retrieval augmented generation

In [17]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://pubmed.ncbi.nlm.nih.gov/12160836/")
docs = loader.load()

In [18]:
from langchain_community.embeddings import OllamaEmbeddings
embeddings = OllamaEmbeddings()

In [19]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [20]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [21]:
from langchain_core.documents import Document
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

response = retrieval_chain.invoke({"input": "Please explain the concept of stemness to me!"})
print(response["answer"])



Stemness is a critical property of stem cells, which refers to their ability to maintain a state of self-renewal and multipotency. In other words, stem cells have the capacity to divide and give rise to more stem cells or differentiated cells, while retaining their undifferentiated state.

Stemness is characterized by several key features:

1. Self-renewal: Stem cells can divide to produce more stem cells, maintaining their population size without differentiating.
2. Multipotency: Stem cells have the ability to give rise to multiple cell types or lineages, such as blood cells, nerve cells, and muscle cells.
3. Plasticity: Stem cells can change their fate in response to changes in their environment or exposure to different growth factors.
4. Quiescence: Stem cells can enter a state of dormancy, which allows them to survive under adverse conditions.
5. Slow cycling: Stem cells divide at a slower rate compared to other cell types, which allows them to maintain their undifferentiated stat

## Use RAG for getting information about a specific article

In [22]:
result = llm.invoke("What did Lutz Leichsenring tell the German broadcaster DW?")
print(result)


I'm just an AI, I don't have access to real-time news or information, and I cannot provide a quote from Lutz Leichsenring as he may have made the statement in the past, but it is not current news. Additionally, I cannot verify the accuracy of any statements attributed to him without further context. If you can provide more information or clarify the context in which you heard this quote, I would be happy to help you further.


In [23]:
loader = WebBaseLoader("https://www.theguardian.com/world/2024/mar/15/berlins-techno-scene-added-to-unesco-intangible-cultural-heritage-list")
docs = loader.load()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [24]:
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

response = retrieval_chain.invoke({"input": "What did Lutz Leichsenring tell the German broadcaster DW?"})
print(response["answer"])


Lutz Leichsenring told the German broadcaster DW that the decision to add Berlin's techno scene to Germany's list of intangible cultural heritage will help ensure that club culture is recognized as a valuable sector worthy of protection and support.
